In [1]:
import numpy as np
import pandas as pd

In [59]:
data_allforms_training = pd.read_csv('../PROACT/all_forms_PROACT_training.txt', sep='|', header=0, dtype={"feature_value":str, "feature_unit":str}, low_memory = False)
data_allforms_training2 = pd.read_csv('../PROACT/all_forms_PROACT_training2.txt', sep='|', header=0, dtype={"feature_value":str, "feature_unit":str}, low_memory = False)
data_allforms_leaderboard = pd.read_csv('../PROACT/all_forms_PROACT_leaderboard_full.txt', sep='|', header=0, dtype={"feature_value":str, "feature_unit":str}, low_memory = False)
data_allforms_validation = pd.read_csv('../PROACT/all_forms_PROACT_validation_full.txt', sep='|', header=0, dtype={"feature_value":str, "feature_unit":str}, low_memory = False)

In [60]:
data_allforms = pd.concat([data_allforms_training, data_allforms_training2, data_allforms_leaderboard, data_allforms_validation])

In [61]:
extent = list(data_allforms["SubjectID"].unique())

In [62]:
print(len(data_allforms["SubjectID"].unique()))

10723


Extract Target variable

In [63]:
ALSFRS_slope = pd.read_csv('../PROACT/ALSFRS_slope.csv')
ALSFRS_slope.query("SubjectID == {0}".format(extent), inplace=True)
extent = list(ALSFRS_slope["SubjectID"].unique())

print(len(ALSFRS_slope["SubjectID"].unique()))

3096


Extract Static Features

In [64]:
demographic = pd.read_csv('../PROACT/demographic.csv')
family_hx = pd.read_csv('../PROACT/family_hx.csv')
als_hx = pd.read_csv('../PROACT/als_hx.csv')
riluzole = pd.read_csv('../PROACT/riluzole.csv')
treatment_group = pd.read_csv('../PROACT/treatment_group.csv')

In [65]:
demographic.query("SubjectID == {0}".format(extent), inplace=True)
family_hx.query("SubjectID == {0}".format(extent), inplace=True)
als_hx.query("SubjectID == {0}".format(extent), inplace=True)
riluzole.query("SubjectID == {0}".format(extent), inplace=True)
treatment_group.query("SubjectID == {0}".format(extent), inplace=True)

In [66]:
print(len(demographic["SubjectID"].unique()))
print(len(family_hx["SubjectID"].unique()))
print(len(als_hx["SubjectID"].unique()))
print(len(riluzole["SubjectID"].unique()))
print(len(treatment_group["SubjectID"].unique()))

3096
708
1653
2648
2910


In [67]:
static = pd.DataFrame(columns=['SubjectID'])
static_list = [demographic, family_hx, als_hx, riluzole, treatment_group]
for i in static_list :
    df = i
    static = static.merge(df, on='SubjectID', how='outer')

Extract Time-resolved Features

In [130]:
alsfrs_3mo = pd.read_csv('../PROACT/alsfrs_total_3mo_meta_slope.csv')
alsfrs_detail_3mo = pd.read_csv('../PROACT/ALSFRS_original.csv')
fvc_3mo = pd.read_csv('../PROACT/fvc_3mo_meta.csv')
svc_3mo = pd.read_csv('../PROACT/svc_3mo_meta.csv')
vitals = pd.read_csv('../PROACT/vitals.csv')
weight = pd.read_csv('../PROACT/weight.csv')
bmi = pd.read_csv('../PROACT/bmi.csv')
lab = pd.read_csv('../PROACT/lab.csv')

In [131]:
alsfrs_3mo.query("SubjectID == {0}".format(extent), inplace=True)
alsfrs_detail_3mo.query("SubjectID == {0}".format(extent), inplace=True)
fvc_3mo.query("SubjectID == {0}".format(extent), inplace=True)
svc_3mo.query("SubjectID == {0}".format(extent), inplace=True)
vitals.query("SubjectID == {0}".format(extent), inplace=True)
weight.query("SubjectID == {0}".format(extent), inplace=True)
bmi.query("SubjectID == {0}".format(extent), inplace=True)
lab.query("SubjectID == {0}".format(extent), inplace=True)

In [132]:
alsfrs_detail_3mo.query('feature_delta < 92', inplace=True)
vitals.query('feature_delta < 92', inplace=True)
weight.query('feature_delta < 92', inplace=True)
bmi.query('feature_delta < 92', inplace=True)
lab.query('feature_delta < 92', inplace=True)

In [133]:
print(len(alsfrs_3mo["SubjectID"].unique()))
print(len(alsfrs_detail_3mo["SubjectID"].unique()))
print(len(fvc_3mo["SubjectID"].unique()))
print(len(svc_3mo["SubjectID"].unique()))
print(len(vitals["SubjectID"].unique()))
print(len(weight["SubjectID"].unique()))
print(len(bmi["SubjectID"].unique()))
print(len(lab["SubjectID"].unique()))

3096
3096
2568
526
2716
2890
1966
2872


In [134]:
alsfrs_3mo.set_index('SubjectID', inplace=True)
alsfrs_detail_3mo.set_index('SubjectID', inplace=True)
fvc_3mo.set_index('SubjectID', inplace=True)
svc_3mo.set_index('SubjectID', inplace=True)
vitals.set_index('SubjectID', inplace=True)
weight.set_index('SubjectID', inplace=True)
bmi.set_index('SubjectID', inplace=True)
lab.set_index('SubjectID', inplace=True)

In [135]:
alsfrs_3mo = alsfrs_3mo.iloc[:,:4]
fvc_3mo = fvc_3mo.iloc[:,:4]
svc_3mo = svc_3mo.iloc[:,:4]

Preprocessing time-resolved data

1) Weight

In [137]:
weight_s1 = weight.iloc[:,[0]].groupby('SubjectID',group_keys=False).agg(['mean', 'max', 'min'])

In [138]:
weight_s2 = weight.groupby('SubjectID').agg(['first', 'last'])
weight_s2['weight_change'] = weight_s2.iloc[:,1] - weight_s2.iloc[:,0]
weight_s2['interval'] = weight_s2.iloc[:,3] - weight_s2.iloc[:,2]
weight_s2['weight_slope'] = weight_s2['weight_change']/weight_s2['interval']

In [139]:
weight_stat = weight_s1.merge(weight_s2[['weight_slope']], on = 'SubjectID', how='outer')
weight_stat.columns = weight_stat.columns.droplevel()
weight_stat.columns = ['mean_weight', 'max_weight', 'min_weight', 'slope_weight']

In [140]:
print("mean value of mean_weight is "+str(weight_stat["mean_weight"].mean()))
print("mean value of max_weight is "+str(weight_stat["max_weight"].mean()))
print("mean value of min_weight is "+str(weight_stat["min_weight"].mean()))
print("mean slope of weight is "+str(weight_stat["slope_weight"].mean()))

mean value of mean_weight is 77.03475513107257
mean value of max_weight is 78.12524814311423
mean value of min_weight is 75.87883804152222
mean slope of weight is 0.0016672242807957952


2) BMI

In [141]:
bmi_stat = bmi.iloc[:,[0]].groupby('SubjectID').mean()
bmi_stat.columns = ['mean_BMI']

In [142]:
print("mean value of mean_bmi is "+str(bmi_stat["mean_BMI"].mean()))

mean value of mean_bmi is 0.0026153509906531574


3) Vitals

In [143]:
vitals_s1 = vitals.iloc[:, [1,2,3,4,5]].groupby('SubjectID').agg(['mean','max', 'min'])
vitals_s1.columns = vitals_s1.columns.droplevel()
vitals_s1.columns = ['mean_bp_d','max_bp_d', 'min_bp_d', 'mean_bp_s', 'max_bp_s', 'min_bp_s', 'mean_pulse', 'max_pulse', 'min_pulse', 'mean_resp_rate', 'max_resp_rate','min_resp_rate', 'mean_temp', 'max_temp', 'min_temp']

In [144]:
vitals_s2 = vitals.groupby('SubjectID').agg(['first', 'last'])

In [145]:
vitals_s2['interval'] = vitals_s2.iloc[:,1] - vitals_s2.iloc[:,0]
vitals_s2['bp_d_slope'] = (vitals_s2.iloc[:,3] - vitals_s2.iloc[:,2])/vitals_s2['interval']
vitals_s2['bp_s_slope'] = (vitals_s2.iloc[:,5] - vitals_s2.iloc[:,4])/vitals_s2['interval']
vitals_s2['pulse_slope'] = (vitals_s2.iloc[:,7] - vitals_s2.iloc[:,6])/vitals_s2['interval']
vitals_s2['resp_rate_slope'] = (vitals_s2.iloc[:,9] - vitals_s2.iloc[:,8])/vitals_s2['interval']
vitals_s2['temp_slope'] = (vitals_s2.iloc[:,11] - vitals_s2.iloc[:,10])/vitals_s2['interval']

In [146]:
vitals_s3 = vitals_s2.iloc[:,13:]
vitals_s3.columns = vitals_s3.columns.droplevel()
vitals_s3.columns = ['bp_d_slope', 'bp_s_slope', 'pulse_slope', 'resp_rate_slope', 'temp_slope']

In [147]:
vitals_stat = vitals_s1.merge(vitals_s3, on='SubjectID', how='outer')

In [148]:
print("mean slope of bp_diastolic is "+str(vitals_stat["bp_d_slope"].mean()))
print("mean slope of bp_systolic is "+str(vitals_stat["bp_s_slope"].mean()))
print("mean slope of pulse is "+str(vitals_stat["pulse_slope"].mean()))
print("mean slope of repiratory_rate is "+str(vitals_stat["resp_rate_slope"].mean()))
print("mean slope of temperature is "+str(vitals_stat["temp_slope"].mean()))

mean slope of bp_diastolic is -0.008871548250814057
mean slope of bp_systolic is -0.027735979126013287
mean slope of pulse is 0.01601237512423807
mean slope of repiratory_rate is 0.002698322055355942
mean slope of temperature is -2.1648143677609954e-05


4) Lab

In [149]:
lab.columns

Index(['feature_delta', 'Absolute Lymphocyte Count', 'Absolute Monocyte Count',
       'Absolute Neutrophil Count', 'C-Reactive Protein', 'CK', 'Creatinine',
       'GFR', 'HDL', 'LDL', 'Total Cholesterol', 'Triglycerides', 'Uric Acid',
       'Urine Creatinine', 'Urine Creatinine Clearance'],
      dtype='object')

In [150]:
lab.drop(columns = ["C-Reactive Protein", "GFR", "HDL", "LDL", "Urine Creatinine", "Urine Creatinine Clearance"], inplace = True)

In [151]:
lab.columns

Index(['feature_delta', 'Absolute Lymphocyte Count', 'Absolute Monocyte Count',
       'Absolute Neutrophil Count', 'CK', 'Creatinine', 'Total Cholesterol',
       'Triglycerides', 'Uric Acid'],
      dtype='object')

In [152]:
lab = lab.astype({'CK': 'float', 'Creatinine': 'float'})
lab_s1 = lab.iloc[:, 1:].groupby('SubjectID').agg(['mean','max', 'min'])
lab_s1.columns = lab_s1.columns.droplevel()
lab_s1.columns = ['mean_ALC','max_ALC', 'min_ALC', 'mean_AMC', 'max_AMC', 'min_AMC', 'mean_ANC', 'max_ANC', 'min_ANC', 'mean_CK', 'max_CK','min_CK', 'mean_Creatinine', 'max_Creatinine', 'min_Creatinine', 'mean_Total_Cholesterol', 'max_Total_Cholesterol', 'min_Total_Cholesterol', 'mean_Triglycerides', 'max_Triglycerides', 'min_Triglycerides', 'mean_Uric_Acid', 'max_Uric_Acid', 'min_Uric_Acid']

In [153]:
lab_s2 = lab.groupby('SubjectID').agg(['first', 'last'])

In [154]:
lab_s2['interval'] = lab_s2.iloc[:,1] - lab_s2.iloc[:,0]
lab_s2['ALC_slope'] = (lab_s2.iloc[:,3] - lab_s2.iloc[:,2])/lab_s2['interval']
lab_s2['AMC_slope'] = (lab_s2.iloc[:,5] - lab_s2.iloc[:,4])/lab_s2['interval']
lab_s2['ANC_slope'] = (lab_s2.iloc[:,7] - lab_s2.iloc[:,6])/lab_s2['interval']
lab_s2['CK_slope'] = (lab_s2.iloc[:,9] - lab_s2.iloc[:,8])/lab_s2['interval']
lab_s2['Creaetinine_slope'] = (lab_s2.iloc[:,11] - lab_s2.iloc[:,10])/lab_s2['interval']
lab_s2['Total_Cholesterol_slope'] = (lab_s2.iloc[:,13] - lab_s2.iloc[:,12])/lab_s2['interval']
lab_s2['Triglycerides_slope'] = (lab_s2.iloc[:,15] - lab_s2.iloc[:,14])/lab_s2['interval']
lab_s2['Uric_Acid_slope'] = (lab_s2.iloc[:,17] - lab_s2.iloc[:,16])/lab_s2['interval']

In [155]:
lab_s3 = lab_s2.iloc[:,19:]
lab_s3.columns = lab_s3.columns.droplevel()
lab_s3.columns = ['ALC_slope', 'AMC_slope' ,'ANC_slope', 'CK_slope', 'Creaetinine_slope', 'Total_Cholesterol_slope', 'Triglycerides_slope','Uric_Acid_slope']

In [127]:
lab_stat = lab_s1.merge(lab_s3, on='SubjectID', how='outer')

5) ALSFRS_3mo

In [157]:
print("mean slope of total alsfrs is "+str(alsfrs_3mo["alsfrs_total_slope"].mean()))
print("mean value of mean_alsfrs_total is "+str(alsfrs_3mo["mean_alsfrs_total"].mean()))
print("mean value of max_alsfrs_total is "+str(alsfrs_3mo["max_alsfrs_total"].mean()))
print("mean value of min_alsfrs_total is "+str(alsfrs_3mo["min_alsfrs_total"].mean()))

mean slope of total alsfrs is -0.01810708927893637
mean value of mean_alsfrs_total is 30.479688845822555
mean value of max_alsfrs_total is 31.66795865633075
mean value of min_alsfrs_total is 29.233527131782946


6) ALSFRS_detail_3mo

In [169]:
alsfrs_detail_3mo = alsfrs_detail_3mo[['feature_delta', 'mouth', 'hands', 'leg', 'trunk', 'respiratory']]

In [170]:
alsfrs_detail_3mo_s1 = alsfrs_detail_3mo.iloc[:, 1:].groupby('SubjectID').agg(['mean','max', 'min'])
alsfrs_detail_3mo_s1.columns = alsfrs_detail_3mo_s1.columns.droplevel()
alsfrs_detail_3mo_s1.columns = ['mean_alsfrs_mouth', 'max_alsfrs_mouth','min_alsfrs_mouth', 'mean_alsfrs_hands', 'max_alsfrs_hands','min_alsfrs_hands', 'mean_alsfrs_leg', 'max_alsfrs_leg','min_alsfrs_leg', 'mean_alsfrs_trunk', 'max_alsfrs_trunk','min_alsfrs_trunk', 'mean_alsfrs_respiratory', 'max_alsfrs_respiratory','min_alsfrs_respiratory']

In [171]:
alsfrs_detail_3mo_s2 = alsfrs_detail_3mo.groupby('SubjectID').agg(['first', 'last'])

In [172]:
alsfrs_detail_3mo_s2['interval'] = alsfrs_detail_3mo_s2.iloc[:,1] - alsfrs_detail_3mo_s2.iloc[:,0]
alsfrs_detail_3mo_s2['alsfrs_mouth_slope'] = (alsfrs_detail_3mo_s2.iloc[:,3] - alsfrs_detail_3mo_s2.iloc[:,2])/alsfrs_detail_3mo_s2['interval']
alsfrs_detail_3mo_s2['alsfrs_hands_slope'] = (alsfrs_detail_3mo_s2.iloc[:,5] - alsfrs_detail_3mo_s2.iloc[:,4])/alsfrs_detail_3mo_s2['interval']
alsfrs_detail_3mo_s2['alsfrs_leg_slope'] = (alsfrs_detail_3mo_s2.iloc[:,7] - alsfrs_detail_3mo_s2.iloc[:,6])/alsfrs_detail_3mo_s2['interval']
alsfrs_detail_3mo_s2['alsfrs_trunk_slope'] = (alsfrs_detail_3mo_s2.iloc[:,9] - alsfrs_detail_3mo_s2.iloc[:,8])/alsfrs_detail_3mo_s2['interval']
alsfrs_detail_3mo_s2['alsfrs_respiratory_slope'] = (alsfrs_detail_3mo_s2.iloc[:,11] - alsfrs_detail_3mo_s2.iloc[:,10])/alsfrs_detail_3mo_s2['interval']

In [178]:
alsfrs_detail_3mo_s3 = alsfrs_detail_3mo_s2.iloc[:,13:]
alsfrs_detail_3mo_s3.columns = alsfrs_detail_3mo_s3.columns.droplevel(1)

In [181]:
alsfrs_detail_3mo_stat = alsfrs_detail_3mo_s1.merge(alsfrs_detail_3mo_s3, on='SubjectID', how='outer')

7) FVC, SVC 3mo

In [183]:
print("mean slope of fvc is "+str(fvc_3mo["fvc_slope"].mean()))
print("mean value of mean_fvc is "+str(fvc_3mo["fvc_mean"].mean()))
print("mean value of max_fvc is "+str(fvc_3mo["fvc_max"].mean()))
print("mean value of min_fvc is "+str(fvc_3mo["fvc_min"].mean()))

mean slope of fvc is -0.06263292919876232
mean value of mean_fvc is 83.69979880581522
mean value of max_fvc is 88.35825545171339
mean value of min_fvc is 78.86059190031153


In [185]:
print("mean slope of svc is "+str(svc_3mo["svc_slope"].mean()))
print("mean value of mean_svc is "+str(svc_3mo["svc_mean"].mean()))
print("mean value of max_svc is "+str(svc_3mo["svc_max"].mean()))
print("mean value of min_svc is "+str(svc_3mo["svc_min"].mean()))

mean slope of svc is -0.07592307077535818
mean value of mean_svc is 88.52043726235745
mean value of max_svc is 93.75475285171103
mean value of min_svc is 83.00190114068441


combine time-resolved features into one dataframe

In [186]:
time_res = pd.DataFrame(columns=['SubjectID'])
time_res_list = [alsfrs_3mo, alsfrs_detail_3mo_stat, fvc_3mo, svc_3mo, vitals_stat, weight_stat, bmi_stat, lab_stat]
for i in time_res_list :
    df = i
    time_res = time_res.merge(df, on='SubjectID', how='outer')

Merge all features into one dataframe

In [188]:
features_combined = static.merge(time_res, on='SubjectID', how='outer')

In [193]:
features_combined.to_csv('../예비4 - 변수선택용 proact 데이터/features_combined_with_nans.csv')

Merge all feauture and target 

In [194]:
features_target_combined = features_combined.merge(ALSFRS_slope, on='SubjectID', how='outer')

In [196]:
features_target_combined.to_csv('../예비4 - 변수선택용 proact 데이터/features_target_combined_with_nans.csv')